In [12]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
 
init_notebook_mode(connected=True)
from sklearn.cluster import KMeans
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot

In [3]:
data = pd.read_excel(r'standarized_data.xlsx')

In [4]:
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].astype('category')
        data[col] = data[col].cat.codes

data['type_of_breast_surgery'] = data['type_of_breast_surgery'].astype('category')
data['type_of_breast_surgery'] = data['type_of_breast_surgery'].cat.codes

In [5]:
from sklearn.decomposition import PCA

def apply_pca(data, columns_to_exclude=None, n_components=3):
    if columns_to_exclude:
        exclude_filter = ~data.columns.isin(columns_to_exclude)
    else:
        exclude_filter = ~data.columns.isin([])

    # Stosowanie PCA na wyfiltrowanych danych
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(data.loc[:, exclude_filter])

    return principal_components

# Używanie zmodyfikowanej funkcji na danych
principal_components = apply_pca(data, columns_to_exclude=['patient_id'])

In [6]:
principal_df = pd.DataFrame(data = principal_components, 
                           columns = ['principal component 1', 'principal component 2', 'principal component 3'])

principal_df['patient_id'] = data['patient_id']
principal_df

,principal component 1,principal component 2,principal component 3,patient_id
0,41.248819,-29.222147,-11.015307,2
1,182.903919,-37.752903,-12.136192,8
2,117.663245,-34.254380,-9.394124,10
3,-17.998712,102.414233,-54.930725,28
4,-55.788066,-18.100330,-12.292287,35
...,...,...,...,...
1514,-61.643444,-22.812678,-7.894099,7278
1515,-51.500579,16.861775,-24.588897,7280
1516,-52.643898,-23.788885,-10.930897,7284
1517,-51.781628,-10.325266,28.693617,7286


In [7]:
trace0 = go.Scatter(
    x = principal_df['principal component 1'],
    y = principal_df['principal component 2'],
    text=principal_df['patient_id'],
    textposition="top center",
    name = 'Piony',
    mode = 'markers+text',
    marker = dict(
        size = 10,
        color = 'rgb(228,26,28)',
        line = dict(
            width = 1,
            color = 'rgb(0, 0, 0)'
        )
    )
)
 
data = [trace0]
 
layout = dict(title = 'PCA',
              yaxis = dict(zeroline = False, title ='PC2 (principal component 2)'),
              xaxis = dict(zeroline = False, title ='PC1 (principal component 1)')
             )
 
fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-scatter')

In [10]:
trace0 = go.Scatter(
    x = principal_df['principal component 2'],
    y = principal_df['principal component 3'],
    text=principal_df['patient_id'],
    textposition="top center",
    name = 'Piony',
    mode = 'markers+text',
    marker = dict(
        size = 10,
        color = 'rgb(228,26,28)',
        line = dict(
            width = 1,
            color = 'rgb(0, 0, 0)'
        )
    )
)
 
data = [trace0]
 
layout = dict(title = 'PCA',
              yaxis = dict(zeroline = False, title ='PC3 (principal component 3)'),
              xaxis = dict(zeroline = False, title ='PC2 (principal component 2)')
             )
 
fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-scatter')

In [11]:
trace0 = go.Scatter(
    x = principal_df['principal component 1'],
    y = principal_df['principal component 3'],
    text=principal_df['patient_id'],
    textposition="top center",
    name = 'Piony',
    mode = 'markers+text',
    marker = dict(
        size = 10,
        color = 'rgb(228,26,28)',
        line = dict(
            width = 1,
            color = 'rgb(0, 0, 0)'
        )
    )
)
 
data = [trace0]
 
layout = dict(title = 'PCA',
              yaxis = dict(zeroline = False, title ='PC3 (principal component 3)'),
              xaxis = dict(zeroline = False, title ='PC1 (principal component 1)')
             )
 
fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-scatter')

### Bardziej przejrzysty plot (mam nadzieję)

In [8]:
trace0 = go.Scatter(
    x = principal_df['principal component 1'],
    y = principal_df['principal component 2'],
    text=principal_df['patient_id'],
    hoverinfo='text',  # Pokazuje identyfikatory pacjentów tylko po najechaniu
    mode = 'markers',
    marker = dict(
        size = 7,  # Mniejsze markery
        color = 'rgb(228,26,28)',
        opacity=0.5,  # Półprzezroczyste markery
        line = dict(
            width = 1,
            color = 'rgb(0, 0, 0)'
        )
    )
)

data = [trace0]

layout = dict(title = 'PCA',
              yaxis = dict(zeroline = False, title ='PC2 (principal component 2)'),
              xaxis = dict(zeroline = False, title ='PC1 (principal component 1)')
             )

fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-scatter')


### KMeans

In [13]:
from sklearn.cluster import KMeans
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot

# Załaduj dane PCA do DataFrame, jeśli jeszcze tego nie zrobiłeś
# principal_df = pd.DataFrame(data = principal_components, ...)

# Określ liczbę klastrów
num_clusters = 5  # Możesz dostosować tę wartość

# Przeprowadź klasteryzację K-means
kmeans = KMeans(n_clusters=num_clusters)
principal_df['cluster'] = kmeans.fit_predict(principal_df[['principal component 1', 'principal component 2']])

# Przygotuj dane do wykresu
traces = []
for cluster_id in range(num_clusters):
    cluster_data = principal_df[principal_df['cluster'] == cluster_id]
    trace = go.Scatter(
        x = cluster_data['principal component 1'],
        y = cluster_data['principal component 2'],
        mode = 'markers',
        marker = dict(
            size = 10,
            opacity = 0.5,  # Półprzezroczyste markery
            line = dict(
                width = 1,
                color = 'rgb(0, 0, 0)'
            )
        ),
        name = f'Cluster {cluster_id}'
    )
    traces.append(trace)

# Dodaj centroidy jako oddzielne punkty
centroids = kmeans.cluster_centers_
centroid_trace = go.Scatter(
    x = centroids[:, 0],
    y = centroids[:, 1],
    mode = 'markers+text',
    marker = dict(
        size = 14,
        color = 'rgb(0,0,0)',
        symbol = 'x'
    ),
    text = [f'Centroid {i}' for i in range(num_clusters)],
    textposition = 'top center',
    name = 'Centroids'
)
traces.append(centroid_trace)

# Ustawienia wykresu
layout = go.Layout(
    title = 'Klasteryzacja PCA z K-means',
    xaxis = dict(title = 'PC1 (principal component 1)'),
    yaxis = dict(title = 'PC2 (principal component 2)'),
    hovermode = 'closest'
)

# Wygeneruj wykres
fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename='pca-kmeans-clustering')


c:\Users\Julia\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

